In [1]:
import sys
from pathlib import Path


try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB and Path.cwd().name == "content":
    if not Path("quantum-jam-chirimbolo").exists():
        !git clone https://github.com/segusantos/quantum-jam-chirimbolo.git
    sys.path.append("quantum-jam-chirimbolo")
    !pip install uv
    !uv sync --locked

Cloning into 'quantum-jam-chirimbolo'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 37 (delta 15), reused 22 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 75.07 KiB | 3.41 MiB/s, done.
Resolving deltas: 100% (15/15), done.


AttributeError: module 'sys' has no attribute 'append'

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector, plot_bloch_vector
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter, ParameterVector
import qiskit.qasm3
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator, QiskitRuntimeService

In [ ]:
from src.test import test

test()